In [193]:
import pandas as pd
import re
import joblib
import numpy as np
from sklearn.model_selection import learning_curve

In [194]:
# set display options
pd.options.display.max_columns = 100

In [195]:
# gen ops - drop_duples,  date, sort
df = pd.read_csv('c:\work\dev\python\progs\scraper\items.csv')
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
df.drop_duplicates(subset=['Event','Fighter_left', 'Fighter_right', 'Date'], inplace=True)
df.sort_values(by='Date', inplace=True)
# df.to_csv('c:\work\dev\python\progs\scraper\items.csv', index=False)

In [196]:
# get judges cards
df['Details:'] = df['Details:'].str.replace(r'\s{2,}', r'_', regex=True)
df['Details:'] = df['Details:'].str.replace(r'\._', r'.', regex=True)
df['Details:'] = df['Details:'].str.replace(r'\.;', r'.', regex=True)

df['judges'] = df['Details:'].str.findall(r"(?P<judge>[A-Za-z'\s]+)[_:]{1}(?P<f1>\d+)\s*-\s*(?P<f2>\d+)")

df.loc[df['judges'].str.len()>0, 'judge1'] = df.loc[df['judges'].str.len()>0, 'judges'].str[0]
df.loc[df['judges'].str.len()>0, 'judge2'] = df.loc[df['judges'].str.len()>0, 'judges'].str[1]
df.loc[df['judges'].str.len()>0, 'judge3'] = df.loc[df['judges'].str.len()>0, 'judges'].str[2]

In [197]:
# rename cols and make fighter-opponent table
df_add = df.copy()
df = df.rename(columns={'Fighter_left':'Fighter', 'Fighter_right':'Opponent', 'Win_lose_left':'Result'}).drop('Win_lose_right', axis=1)

def suf_change(col_names, suf_out, suf_dam):
    cols = []
    
    for it in col_names:
        if re.search(r'{suf}$'.format(suf=suf_dam), it):
            cols.append(re.sub(r'{suf}$'.format(suf=suf_dam), r'_d', it))
        elif re.search(r'{suf}((_)|$)'.format(suf=suf_out), it):
            cols.append(re.sub(r'{suf}((_)|$)'.format(suf=suf_out), r'\1', it))
        elif re.search(r'{suf}(_)'.format(suf=suf_dam), it):
            cols.append(re.sub(r'{suf}(_)'.format(suf=suf_dam), r'_d\1', it))
        else:
            cols.append(it)
    return cols

df.columns = suf_change(df.columns, suf_out='_l', suf_dam = '_r')

df_add = df_add.rename(columns={'Fighter_right':'Fighter', 'Fighter_left':'Opponent', 'Win_lose_right':'Result'}).drop('Win_lose_left', axis=1)
df_add.columns = suf_change(df_add.columns, suf_out='_r', suf_dam = '_l')

df_all = pd.concat([df,df_add], ignore_index=True)

del df, df_add

In [198]:
#getting fighter params, adding to all suffix _stat
#already float cols
df_all.rename(columns={'KD':'kd_stat', 'Rev.':'rev_stat', 'Sub. att':'sub_att_stat', 
                       'KD_d':'kd_dam_stat', 'Rev._d':'rev_dam_stat', 'Sub. att_d':'sub_att_dam_stat'}, inplace=True)

df_all['sig_str_stat'] = df_all['Sig. str'].str.extract(r'(\d)\sof', expand=False)
df_all['sig_str_stat'] = df_all['sig_str_stat'].astype(float)

df_all['sig_str_dam_stat'] = df_all['Sig. str_d'].str.extract(r'(\d)\sof', expand=False)
df_all['sig_str_dam_stat'] = df_all['sig_str_dam_stat'].astype(float)

df_all['sig_str_h_stat'] = df_all['Head'].str.extract(r'(\d)\sof', expand=False)
df_all['sig_str_h_stat'] = df_all['sig_str_h_stat'].astype(float)

df_all['sig_str_h_dam_stat'] = df_all['Head_d'].str.extract(r'(\d)\sof', expand=False)
df_all['sig_str_h_dam_stat'] = df_all['sig_str_h_dam_stat'].astype(float)

df_all['sig_str_l_stat'] = df_all['Leg'].str.extract(r'(\d)\sof', expand=False)
df_all['sig_str_l_stat'] = df_all['sig_str_l_stat'].astype(float)

df_all['sig_str_l_dam_stat'] = df_all['Leg_d'].str.extract(r'(\d)\sof', expand=False)
df_all['sig_str_l_dam_stat'] = df_all['sig_str_l_dam_stat'].astype(float)

df_all['sig_str_b_stat'] = df_all['Body'].str.extract(r'(\d)\sof', expand=False)
df_all['sig_str_b_stat'] = df_all['sig_str_b_stat'].astype(float)

df_all['sig_str_b_dam_stat'] = df_all['Body_d'].str.extract(r'(\d)\sof', expand=False)
df_all['sig_str_b_dam_stat'] = df_all['sig_str_b_dam_stat'].astype(float)

df_all['sig_str_d_stat'] = df_all['Distance'].str.extract(r'(\d)\sof', expand=False)
df_all['sig_str_d_stat'] = df_all['sig_str_d_stat'].astype(float)

df_all['sig_str_d_dam_stat'] = df_all['Distance_d'].str.extract(r'(\d)\sof', expand=False)
df_all['sig_str_d_dam_stat'] = df_all['sig_str_d_dam_stat'].astype(float)

df_all['sig_str_cl_stat'] = df_all['Clinch'].str.extract(r'(\d)\sof', expand=False)
df_all['sig_str_cl_stat'] = df_all['sig_str_cl_stat'].astype(float)

df_all['sig_str_cl_dam_stat'] = df_all['Clinch_d'].str.extract(r'(\d)\sof', expand=False)
df_all['sig_str_cl_dam_stat'] = df_all['sig_str_cl_dam_stat'].astype(float)

df_all['sig_str_gr_stat'] = df_all['Ground'].str.extract(r'(\d)\sof', expand=False)
df_all['sig_str_gr_stat'] = df_all['sig_str_gr_stat'].astype(float)

df_all['sig_str_gr_dam_stat'] = df_all['Ground_d'].str.extract(r'(\d)\sof', expand=False)
df_all['sig_str_gr_dam_stat'] = df_all['sig_str_gr_dam_stat'].astype(float)

df_all['td_stat'] = df_all['Td'].str.extract(r'(\d)\sof', expand=False)
df_all['td_stat'] = df_all['td_stat'].astype(float)

df_all['td_dam_stat'] = df_all['Td_d'].str.extract(r'(\d)\sof', expand=False)
df_all['td_dam_stat'] = df_all['td_dam_stat'].astype(float)


In [199]:
# ctrl_stat, time_stat in minutes
df_all['ctrl_stat'] = df_all['Ctrl'].str.extract(r'(\d:\d{2})', expand=False)
df_all.loc[df_all['ctrl_stat'].notnull(),'ctrl_stat'] = df_all.loc[df_all['ctrl_stat'].notnull(),'ctrl_stat'].str.split(':')\
                                                        .map(lambda x: int(x[0]) + int(x[1])/60)
df_all['ctrl_stat'].fillna(0, inplace=True)


df_all['ctrl_dam_stat'] = df_all['Ctrl_d'].str.extract(r'(\d:\d{2})', expand=False)
df_all.loc[df_all['ctrl_dam_stat'].notnull(),'ctrl_dam_stat'] = df_all.loc[df_all['ctrl_dam_stat'].notnull(),'ctrl_dam_stat'].str.split(':')\
                                                        .map(lambda x: int(x[0]) + int(x[1])/60)
df_all['ctrl_dam_stat'].fillna(0, inplace=True)


df_all['time'] = (df_all['Round:']-1)*5 + df_all['Time:'].str.split(':').map(lambda x: int(x[0])+int(x[1])/60)

# normalise stat cols stat to minutes 
stat_cols = [it for it in df_all.columns if re.search('_stat', it)]
for col in stat_cols:
    df_all[col] = df_all[col]/df_all['time']

df_all['win_stat'] = (df_all['Result']=='W').astype(np.int8)
df_all['lose_stat'] = (df_all['Result']=='L').astype(np.int8)

In [200]:
# type of win stat
met_d = {'Decision - Unanimous':'decision', "TKO - Doctor's Stoppage":'KO/TKO', 'KO/TKO':'KO/TKO', 'Submission':'wrestling', 'Decision - Split':'decision',
        'Decision - Majority':'decision', 'Could Not Continue':'KO/TKO', 'Overturned':'other', 'Other':'other', 'DQ':'other'} 

df_all['method_type'] = df_all['Method:'].map(met_d)

df_all['wrest_w_stat'] = ((df_all['method_type']=='wrestling') & (df_all['win_stat'])).astype(np.int8)
df_all['wrest_l_stat'] = ((df_all['method_type']=='wrestling') & (df_all['lose_stat'])).astype(np.int8)

df_all['KO_w_stat'] = ((df_all['method_type']=='KO/TKO')&(df_all['win_stat'])).astype(np.int8)
df_all['KO_l_stat'] = ((df_all['method_type']=='KO/TKO') &(df_all['lose_stat'])).astype(np.int8)

df_all['dec_w_stat' ] = ((df_all['method_type']=='decision') &(df_all['win_stat'])).astype(np.int8)
df_all['dec_l_stat' ] = ((df_all['method_type']=='decision') &(df_all['lose_stat'])).astype(np.int8)

In [201]:
# drop rows where sign strikes is null
df_all = df_all[df_all['Sig. str'].notnull()]
# drop fighters duples 
df_all = df_all[df_all['Fighter']!='Phillip Hawes']


In [202]:
# modify some cols
# for col in [it for it in df_all.columns if re.search('str.*_stat$', it)]:
#     df_all[col] = np.clip(df_all[col], 0, df_all[col].quantile(0.95))
#     df_all[col] = np.power(df_all[col], 0.2)


In [203]:
def get_rolling_stats(df_fighter, op_d, min_fights_num):
#     import pdb; pdb.set_trace()
    df_stat = df_fighter.expanding(min_periods=min_fights_num).agg(op_d)
    df_stat['nlosses_stat_rol'] = df_fighter['lose_stat'].rolling(window=min_fights_num).sum()
    
    return df_stat.shift(1)
#     return df_stat


min_fights_num=3
df_all.sort_values(by=['Fighter', 'Date'], inplace=True)
stat_cols = [it for it in df_all.columns if re.search('_stat', it)]
# op_d = {col:'mean' for col in stat_cols}
# op_d = {col:'median' for col in stat_cols}
op_d = {col:'median' for col in stat_cols if re.search('str',col)}
op_d.update({col:'mean' for col in stat_cols if not re.search('str',col)})

df_stat = df_all.groupby('Fighter').apply(lambda df: get_rolling_stats(df,op_d, min_fights_num))

In [204]:
df_full = df_all.merge(df_stat.reset_index(level=0), left_index=True, right_index=True, suffixes=['','_rol'], how='outer')

In [205]:
joblib.dump(df_all, 'df_all.pkl')
del df_all, df_stat

In [206]:
df_full.to_csv('ufc_excel.csv', index=False, decimal=',')
df_full.to_csv('ufc_datalens.csv', index=False)

# Make sub vectors

In [207]:
df_full = df_full[df_full['kd_stat_rol'].notnull()]
# draw maybe divide on four win/lose of one, win/lose of other
df_full = df_full[df_full.Result.isin(['W', 'L'])]
feat_cols = [it for it in df_full.columns if re.search('_stat_rol', it)]

In [208]:
df1 = df_full.set_index(['Fighter', 'Opponent', 'Date'])[feat_cols]
df2 = df_full.set_index([ 'Opponent','Fighter', 'Date'])[feat_cols]
df2.index.set_names(df1.index.names, inplace=True)

res = df1-df2
res = res[res.kd_stat_rol.notnull()]

In [209]:
df_feat = df_full.merge(res.reset_index(), on = ['Fighter','Opponent','Date'], suffixes=['','_sub'])
df_feat['Result'] = np.where(df_feat['Result']=='W',1,0)

In [210]:
# import joblib;joblib.dump(df_feat, 'df_feat_trick.pkl')
import joblib;joblib.dump(df_feat, 'df_feat_med.pkl')

['df_feat_med.pkl']

# Try to predict

In [211]:
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, accuracy_score

In [212]:
clf = CatBoostClassifier(task_type="GPU", verbose = False, random_seed=0)

In [213]:
sign_cols = [it for it in df_feat.columns if re.search('_stat_rol_sub', it) or it in ['Fighter', 'Opponent', 'Date', 'Result'] ]
X = df_feat[sign_cols]
X_tr = X[X['Date']<pd.Timestamp('2021-09-01')].drop(['Fighter', 'Opponent', 'Date', 'Result'], axis=1)
X_ts = X[X['Date']>=pd.Timestamp('2021-09-01')].drop(['Fighter', 'Opponent', 'Date', 'Result'], axis=1)
y_tr = X.loc[X['Date']<pd.Timestamp('2021-09-01'), 'Result']
y_ts = X.loc[X['Date']>=pd.Timestamp('2021-09-01'), 'Result']

In [214]:
X.shape

(5024, 37)

In [215]:
clf.fit(X_tr, y_tr)

In [216]:
idx = np.argsort(clf.feature_importances_)[::-1]
np.array(clf.feature_names_)[idx]

array(['td_dam_stat_rol_sub', 'ctrl_dam_stat_rol_sub', 'td_stat_rol_sub',
       'sig_str_l_dam_stat_rol_sub', 'sig_str_l_stat_rol_sub',
       'dec_w_stat_rol_sub', 'sig_str_cl_dam_stat_rol_sub',
       'dec_l_stat_rol_sub', 'win_stat_rol_sub', 'kd_dam_stat_rol_sub',
       'sig_str_stat_rol_sub', 'sig_str_gr_dam_stat_rol_sub',
       'sub_att_dam_stat_rol_sub', 'sig_str_h_dam_stat_rol_sub',
       'sig_str_cl_stat_rol_sub', 'sig_str_dam_stat_rol_sub',
       'kd_stat_rol_sub', 'KO_l_stat_rol_sub', 'sig_str_gr_stat_rol_sub',
       'KO_w_stat_rol_sub', 'sig_str_h_stat_rol_sub',
       'rev_dam_stat_rol_sub', 'sig_str_b_dam_stat_rol_sub',
       'wrest_l_stat_rol_sub', 'ctrl_stat_rol_sub', 'lose_stat_rol_sub',
       'rev_stat_rol_sub', 'wrest_w_stat_rol_sub',
       'sig_str_d_dam_stat_rol_sub', 'sub_att_stat_rol_sub',
       'sig_str_b_stat_rol_sub', 'sig_str_d_stat_rol_sub',
       'nlosses_stat_rol_sub'], dtype='<U27')

In [217]:
y_p = clf.predict(X_ts)
y_proba = clf.predict_proba(X_ts)
# y_p = (y_proba[:,1]>0.9).astype(np.uint8)

In [218]:
print(classification_report(y_ts, y_p))

              precision    recall  f1-score   support

           0       0.65      0.65      0.65        48
           1       0.65      0.65      0.65        48

    accuracy                           0.65        96
   macro avg       0.65      0.65      0.65        96
weighted avg       0.65      0.65      0.65        96



In [219]:
print(classification_report(y_tr, clf.predict(X_tr)))

              precision    recall  f1-score   support

           0       0.89      0.90      0.90      2464
           1       0.90      0.89      0.89      2464

    accuracy                           0.89      4928
   macro avg       0.89      0.89      0.89      4928
weighted avg       0.89      0.89      0.89      4928



In [67]:
def draw_learning_curve(estim,x,y,sizes_l, cv=3):
    train_sizes,train_scores, test_scores = learning_curve(estim, x, y, train_sizes=sizes_l,cv=cv,n_jobs=-1)
    train_mean = np.mean(train_scores,axis=1)
    train_std = np.std(train_scores, axis=1)
    test_mean = np.mean(test_scores, axis =1)
    test_std = np.std(test_scores, axis=1)
    plt.plot(train_sizes,train_mean, color='blue', marker='o', markersize=5, label = 'правильность при обучении')
    plt.fill_between(train_sizes, train_mean+train_std , train_mean - train_std,alpha = 0.2,color='blue')
    plt.plot(train_sizes,test_mean, color='green',marker='s', markersize=5, label='правильность при проверке')
    plt.fill_between(train_sizes, test_mean+test_std, test_mean - test_std, alpha=0.2, color = 'green')
    plt.grid()
    plt.xlabel('количество обучающих образцов')
    plt.ylabel('правильность')
    plt.legend()
    plt.ylim([0.5, 1])
    plt.show()

In [ ]:
# draw_learning_curve(clf, X_tr, y_tr, sizes_l=[0.1, 0.5,0.6,0.9, 1])

In [220]:
X_ts['y_p'] = y_p
X_ts['y_proba'] = y_proba[:,1]
df_comp = X.merge(X_ts[['y_p', 'y_proba']], left_index=True, right_index=True)[['Fighter', 'Opponent', 'Date', 'Result','y_p', 'y_proba' ]]
df_comp[df_comp['Date']==df_comp['Date'].max()]
# df_comp

,Fighter,Opponent,Date,Result,y_p,y_proba
158,Alexander Volkov,Marcin Tybura,2021-10-30,1,0,0.495698
221,Amanda Ribas,Virna Jandiroba,2021-10-30,1,0,0.471828
995,Cory Sandhagen,Petr Yan,2021-10-30,0,0,0.349992
1092,Dan Hooker,Islam Makhachev,2021-10-30,0,0,0.366294
1922,Glover Teixeira,Jan Blachowicz,2021-10-30,1,1,0.502545
2053,Islam Makhachev,Dan Hooker,2021-10-30,1,1,0.718802
2180,Jan Blachowicz,Glover Teixeira,2021-10-30,0,0,0.479865
2867,Khamzat Chimaev,Li Jingliang,2021-10-30,1,1,0.858860
2945,Lerone Murphy,Makwan Amirkhani,2021-10-30,1,1,0.501967
2952,Li Jingliang,Khamzat Chimaev,2021-10-30,0,0,0.139450


# global TODO 
- statistic cover
- predict way of win
- proba (win of first-second + loss second-first)/2
- corner color, is_champ add to features
- striks - opponent

# Description

Takedown (TD)
Knockdown (KD)